# Building a Text Message Spam Filter Using the Naive Bayes Algorithm 

In this project, we will be building a spam filter using a multinomial Naive Bayes algorithm to classify whether incoming SMS messages are spam or not.

The dataset used for both training and testing of the algorithm was created by Tiago A. Almeida and José María Gómez Hidalgo, and can be found at [The UCL Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The dataset contains SMS messages that are already classified as being spam or not.

The Naive Bayes algorithm will assess whether each individual SMS message is spam by evaluating the word contents of the message. As the algorithm is 'Naive', it assumes there is conditional independence between the words in the message which allows the following probability relationships to be developed (full derivation of the relationships can be found in the Appendix):

$$P(Spam|w_{1},w_{2},...w_{3}) \ = \ P(Spam) \ \Pi_{i=1}^{n} \ P(w_{i}|Spam)$$

Where:

$w_{i}$ - the ith word of the message

## Summary of Results

The algorithm correctly predicts 98.7% of the test data. The messages which were wrongly predicted contained various elements which may have escaped the algorithm capabilities such as punctual emojis, abbreviations and acronyms.

# Reading in and Exploring the Data

The dataset can be downloaded from [this link](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition). It contains 2 columns, *Label* and *SMS*. *Label* details whether the message is spam or not and contains the following values:

- *spam* - the message is spam
- *ham* - the message is not spam

*SMS* details the contents of the SMS message.

The dataset was randomised and split in a 80/20 proportion into a Training and Test set. Below details the code which reads in the data, and cleans the strings so our data is ready for analysis and algorithm training.

In [1]:
# Import relevant python packages
import pandas as pd
import numpy as np
import re

In [2]:
# Read in the SMS dataset file
spam_sms = pd.read_csv("SMSSpamCollection", sep='\t', header=None, names=['Label', 'SMS']) 
spam_sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
# Check how many rows and columns our data set has
spam_sms.shape

(5572, 2)

In [4]:
# Predict the probability of getting a spam  message from the dataset
spam_sms['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

We've seen that from the dataset, it is predicted that the probability of receiving a spam SMS message is 13%. In order to maintain this probability distribution when we split our dataset for training and test purposes, we will randomise the data set and split accordingly.

In [5]:
# Randomise the entire dataset
spam_sms_random = spam_sms.sample(frac=1, random_state=1)

# Find the row index which encapsulated 80% of the data
training_index = round(len(spam_sms_random) * 0.8)

# Split the data into training and test data sets in a 80/20 split.
training_data = spam_sms_random.iloc[:training_index].reset_index()
test_data = spam_sms_random.iloc[training_index:].reset_index()

Having split the dataset accordingly, we should check that the spam and non-spam probabilities have been maintained.

In [6]:
training_data['Label'].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [7]:
test_data['Label'].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

Exploring the *SMS* column, we can see that the message contents contain punctuation and uppercase letters which will affect consistency when applying the algorithm. To standardise all SMS messages, we will remove punctuation and convert all of the letters to lowercase.

In [8]:
training_data['SMS'].head()

0                         Yep, by the pretty sculpture
1        Yes, princess. Are you going to make me moan?
2                           Welp apparently he retired
3                                              Havent.
4    I forgot 2 ask ü all smth.. There's a card on ...
Name: SMS, dtype: object

In [9]:
# Replace all punctuation in the SMS column using the regex pattern \W
training_data['SMS'] = training_data['SMS'].str.replace(r'\W', ' ')

# Convert all letters to lowercase
training_data['SMS'] = training_data['SMS'].str.lower()

training_data['SMS'].head()

0                         yep  by the pretty sculpture
1        yes  princess  are you going to make me moan 
2                           welp apparently he retired
3                                              havent 
4    i forgot 2 ask ü all smth   there s a card on ...
Name: SMS, dtype: object

# Data Cleaning

To calculate all the probabilities required by the Naive Bayes algorithm, we need to transform the data into a more usable format.

For illustrative purposes, at the moment our training dataset is formatted like this:

| Label | SMS |
| --- | --- |
| Ham | yep by the pretty sculpture |
| Ham | yes  princess  are you going to make me moan |
| Ham | welp apparently he retired |
| ... | ...|

And we want to transform it to this:

| Label | yep | yes | welp | by | princess | ... |
| --- | --- | --- | --- | --- | --- | --- |
| Ham | 1 | 0 | 0 | 1 | 0 | ... |
| Ham | 0 | 1 | 0 | 0 | 1 | ... |
| Ham | 0 | 0 | 1 | 0 | 0 | ... |

Essentially, we want columns of *all* the unique words used in the dataset with corresponding counts of the words per SMS message in each row. All of the unique words used in the dataset will be termed the *vocabulary*. This will help us to calculate the conditional probabilities for each word given a spam or non-spam message, which will be used in the classification algorithm.

To do this we will create a dictionary with key-value pairs being each unique word in the vocabulary with a corresponding list of word counts for each SMS message in the training set.

In [10]:
# Split each message string
training_data['SMS'] = training_data['SMS'].str.split()

# Initiate empty list to store all words used
vocabulary = []

# Loop through the SMS messages in the training dataset, and append the words to the vocabulary list
for msg in training_data['SMS']:
    for word in msg:
        vocabulary.append(word)

# Get the unique words in the vocabulary
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)


In [11]:
# Initiate empty dictionary with keys being the unique words used in the vocabulary, corresponding value being a list of zeros
# with length equal to the total number of SMS messages in the training dataset

word_counts_per_sms = {word: [0] * len(training_data['SMS'])
                       for word in vocabulary}

# fill the value of each unique word key with the word count encountered in each SMS message
for index, sms in enumerate(training_data['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        

In [13]:
# Create a dataframe from the unique word counts dictionary
word_counts_per_sms = pd.DataFrame(word_counts_per_sms)
word_counts_per_sms.head()

,engin,85555,proof,fromwrk,noon,requests,forth,xxx,hui,predicte,...,okey,someone,cab,chip,adore,09,chapel,80160,nok,pity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Concatenate the unique word counts dictionary to the training data set
training_set = pd.concat([training_data, word_counts_per_sms], axis=1)
training_set.head()

,index,Label,SMS,engin,85555,proof,fromwrk,noon,requests,forth,...,okey,someone,cab,chip,adore,09,chapel,80160,nok,pity
0,1078,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4028,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,958,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4642,ham,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Preparing Constants for the Naive Bayes Algorithm

We have now created the training data set with the corresponding word counts per message. In order to build the algorithm we require the following parameters:

- $P(Spam)$ - the probability of a message being spam
- $P(Ham)$ - the probability of a message being non-spam
- $N_{spam}$ - total number of words in spam messages
- $N_{ham}$ - total number of words in non-spam messages
- $N_{vocab}$ - the number of unique words in the vocabulary
- $\alpha \ = 1$ - constant for Laplace smoothing of conditional probability equations (see Appendix)


In [15]:
# Set P(Spam) and P(Ham)
spam_ham = training_set['Label'].value_counts(normalize=True)
p_spam = spam_ham['spam']
p_ham = spam_ham['ham']

In [16]:
# Set total number of words in spam and non-spam messages
n_spam = 0
n_ham = 0

for msg in training_set[training_set['Label'] == 'spam']['SMS']:
    n_spam += len(msg)
for msg in training_set[training_set['Label'] == 'ham']['SMS']:
    n_ham += len(msg)


In [17]:
# Set total number of unique words in the vocabulary
n_vocab = training_set.iloc[:,3:].shape[1]

In [18]:
# Set Laplace smoothing constant
alpha = 1

To calculate $P(Spam|w_{1},w_{2},...w_{3})$ we need to calculate:

$$P(w_{i}|Spam) \ = \ \frac{N_{w_{i}|Spam}+\alpha}{N_{spam}+\alpha N_{vocab}}$$

Where:

- $N_{w_{i}|Spam}$ - the number of times a word appears in spam messages

And vice versa for non-spam messages.

This is the  probability for each word appearing in spam or non-spam messages. It is therefore a constant for each word and spam/non-spam pair.

To do this we will create 2 dictionaries for spam and non-spam messages containing key-value pairs of each unique word in the vocabulary with the corresponding number of times they are appear in spam/non-spam messages.

In [19]:
# Create dictionaries for spam and non-spam messages containing word counts for each word in the vocabulary
unique_words_spam = {word: 0 for word in training_set.columns[3:]}
unique_words_ham = {word: 0 for word in training_set.columns[3:]}

In [20]:
# Isolate the spam and non-spam SMS messages in separate dataframes
spam = training_set[training_set['Label'] == 'spam']
ham = training_set[training_set['Label'] == 'ham']

In [21]:
# Calculate the conditional probability for each word for spam and non-spam messages using the formula above
for word in unique_words_spam:
    unique_words_spam[word] = (spam[word].sum() + alpha) / (n_spam + alpha * n_vocab)

for word in unique_words_ham:
    unique_words_ham[word] = (ham[word].sum() + alpha) / (n_ham + alpha * n_vocab)

# Creating the Algorithm

Below details the function code which encapsulated the Naive Bayes algorithm to classify whether an SMS message is spam or not.

The function will then be used on the test data to discover the accuracy of the classification.

In [22]:
def classify(message):
    
    # Pre cleaning the SMS message string
    
    message = re.sub('\W', ' ', message) # Remove all punctuation from the string
    message = message.lower() # Convert all letters to lowercase
    message = message.split() # Split on whitespace to obtain list of words in the message
    
    # Initiate the probabilities used to classify whether the message is spam or not
    # The variables are set equal to their respective total probability
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    # Loop through the words in the SMS message and obtain their corresponding conditional probability in the
    # spam and non-spam dictionaries defined above
    # If the word is not in the dictionaries, it is ignored as it is therefore not in the model vocabulary
    
    for word in message:
        if word in unique_words_spam:
            p_spam_given_message *= unique_words_spam[word]
        if word in unique_words_ham:
            p_ham_given_message *= unique_words_ham[word]
        else:
            continue
    
    # Return relevant result
    # If the probabilities for spam and non-spam are equal, then the result required human input
    
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [23]:
test_data['predicted'] = test_data['SMS'].apply(classify)

In [24]:
test_data.head()

,index,Label,SMS,predicted
0,2131,ham,Later i guess. I needa do mcat study too.,ham
1,3418,ham,But i haf enuff space got like 4 mb...,ham
2,3424,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,1538,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,5393,ham,"All done, all handed in. Don't know if mega sh...",ham


In [25]:
correct = 0
total = test_data['SMS'].shape[0]

for index, row in test_data.iterrows():
    if row['Label'] == row['predicted']:
        correct += 1

accuracy = correct / total
print('The accuracy of the model is:', 100 * round(accuracy, 3), '%')

The accuracy of the model is: 98.7 %


Now let's investigate where the model wrongly classified the SMS messages.

In [26]:
# define function to return whether the model classified correctly or not
def checker(row):
    if row.Label == row.predicted:
        return 'Correct'
    else:
        return 'Wrong'

test_data['result'] = test_data.apply(checker, axis=1)

In [27]:
# Print the SMS message contents of the messages the algorithm wrongly classified
pd.set_option("max_colwidth", None)
print(test_data[test_data['result'] == 'Wrong']['SMS'])

114                                                                                                                                                                                                                                                                                                         Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net
135                                                                                                                                                                                                                                                                                                   More people are dogging in your area now. Call 09090204448 and join like minded guys. Why not arrange 1 yourself. There's 1 this evening. A£1.50 minAPN LS278BB
152                                                                                         

Inspecting the data above, we can see that the SMS messages that were wrongly classified have several features in common:

- They contain punctual emoticons such as :D, :) or ;)
- They contain abbreviations or slang terms such as ur (your), gv (give) or std (standard)

Our model did not accomodate for these features, and this is possibly why they have been wrongly classified. To further develop the model we could:

- Try to assign common slang terms to their corresponding word and incorporate them into the corresponding parameter
- Try to make the model sensitive to letter case
